## Semantic role labeling

Важные ресурсы, связанные с srl:  

    Verbnet - https://verbs.colorado.edu/~mpalmer/projects/verbnet.html  
    Propbank - https://propbank.github.io/  
    Framenet - https://framenet.icsi.berkeley.edu/  
    Framebank (на русском) - https://github.com/olesar/framebank  
    Conll dataset - http://www.lsi.upc.edu/~srlconll/  

К сожалению они либо под лицензией (нужно что-то заполнять или даже платить), либо так сложно устроены, что непонятно как с ними работать.

Поэтому в семинаре мы будем использовать данные отсюда - https://dada.cs.washington.edu/qasrl/

Это нестандартная коллекция. Тут вместо отметок ролей - вопросы. Это может показаться неправильным, но если подумать, то семантические роли - это как раз ответы на вопросы "кто/что, когда, как, почему". 

Формулировка задачи в вопросно-ответной форме вообще очень актуальная вещь. В saleforce даже сделали нейронку, которая может решать сразу несколько разных задач, сформулированных как вопросы-ответы. Этот датасет там тоже есть - https://decanlp.com/

In [ ]:
import pandas as pd
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

Посмотрим на данные

In [ ]:
data = open('wiki1.train.qa')

In [ ]:
data.readline()

Формат тут неочевидный, поэтому проще почитать документацию - https://dada.cs.washington.edu/qasrl/data/readme.txt

Для начала попробуем распозновать только предикаты.

In [ ]:
## TRAIN SET

sent_pred_pairs_train = defaultdict(list)
data = open('wiki1.train.qa')
line = data.readline()
while line:
    if line.startswith('WIKI'):
        _, pred_n = line.rstrip('\n').split('\t')
        sent = data.readline().rstrip('\n')
        for i in range(int(pred_n)):
            pred_idx, _, n_qs = data.readline().strip('\n').split('\t')
            for j in range(int(n_qs)):
                _= data.readline()
                
                
            sent_pred_pairs_train[sent].append(pred_idx)
    line = data.readline()

In [ ]:
## DEV SET

sent_pred_pairs_dev = defaultdict(list)
data = open('wiki1.dev.qa')
line = data.readline()
while line:
    if line.startswith('WIKI'):
        _, pred_n = line.rstrip('\n').split('\t')
        sent = data.readline().rstrip('\n')
        for i in range(int(pred_n)):
            pred_idx, _, n_qs = data.readline().strip('\n').split('\t')
            for j in range(int(n_qs)):
                _= data.readline()
                
                
            sent_pred_pairs_dev[sent].append(pred_idx)
    line = data.readline()

In [ ]:
sent_pred_pairs_train

Каждому предложению соответсвует один или больше индексов предикатов.

## Задание 1. 
Преобразуйте этот формат в пригодный для обучения классификаторов. Обучите какой-нибудь простой и оцените на отложенной выборке.

Теперь попробуем решить задачу целиком.

In [ ]:
qa_pairs_train = []
data = open('wiki1.train.qa')
line = data.readline()
while line:
    if line.startswith('WIKI'):
        _, pred_n = line.rstrip('\n').split('\t')
        sent = data.readline()
        for i in range(int(pred_n)):
            pred_idx, _, n_qs = data.readline().strip('\n').split('\t')
            for j in range(int(n_qs)):
                q, a = data.readline().split('?')
                q = ' '.join([w for w in q.split('\t') if w !='_'])
                answers = a.lstrip('\t').rstrip('\n').split('###')
                qa_pairs_train.append((sent, pred_idx, q, answers))
    line = data.readline()

In [ ]:
qa_pairs_dev = []
data = open('wiki1.dev.qa')
line = data.readline()
while line:
    if line.startswith('WIKI'):
        _, pred_n = line.rstrip('\n').split('\t')
        sent = data.readline()
        for i in range(int(pred_n)):
            pred_idx, _, n_qs = data.readline().strip('\n').split('\t')
            for j in range(int(n_qs)):
                q, a = data.readline().split('?')
                
                q = ' '.join([w for w in q.split('\t') if w !='_'])
                answers = a.lstrip('\t').rstrip('\n').split('###')
                qa_pairs_dev.append((sent, pred_idx, q, answers))
    line = data.readline()

Чтобы сопоставить ответы и предложения нам понадобится вот такая функция. Она находит, где в строке встречается ответ, а затем разбивает на токены, сохраняя отметки ответов. Если ответ состоит из нескольких токенов, то они маркируются BI тэгами.

In [ ]:
import re
import string
def label_text(text, answers, tag):
    labels = []
    text = re.sub('  +', ' ', text)

    for word in answers:
        word = word.strip()
        start = text.find(word)
        if start >= 0:
            labels.append((start, start+len(word)))
    
    
    words = text.split()
    if not labels:
        return [(word, 'O') for word in words]
    
    spans = []
    i = 0
    for word in words:
        strip_word_right = word.rstrip(string.punctuation)
        strip_word_left = word.lstrip(string.punctuation)

        spans.append((i, i+len(word)-len(strip_word_left), i+len(word), i+len(strip_word_right)))
        i += len(word)
        i += 1

    tags = []
    for span in spans:
        for label in labels:
            if (span[0] >= label[0] or span[1] >= label[0]) \
              and (span[2] <= label[1] or span[3] <= label[1]):
                tags.append(tag)
                break
        else:
            tags.append('O')
    bio_tags = []
    inside = False
    for tag in tags:
        if tag != 'O':
            if inside:
                bio_tags.append(tag+'-I')
            else:
                bio_tags.append(tag+'-B')
                inside = True
        else:
            bio_tags.append(tag)
            inside = False
    
    return list(zip(words, bio_tags))

In [ ]:
qa_pairs_train[0][-1]

In [ ]:
qa_pairs_train[0][2]

Проще всего понять как это работает на примере.

In [ ]:
label_text(qa_pairs_train[0][0], qa_pairs_train[0][-1], 'ANSWER')

Вообще эта задача требует сложных методов решения, таких как нейронки. Но мы попробуем решить её стандартными подходами.

Преобразуем все в обучающую выборку такого формата - каждый строка это текущее слово, которому нужно предсказать тэг. В качестве признаков будем использовать ещё предыдущее слово, предыдущий тэг и вопрос.

In [ ]:
train_words = []
train_tags = []
train_previous_words = []
train_previous_tags = []
train_questions = []

for pair in qa_pairs_train:
    sent = pair[0].replace(' , ',  ', ')
    answers = pair[-1]
    q = pair[2]
    
    tagged = [('<START>', '<START>'), ] + label_text(sent, answers, 'ANS')
    if not any([tag.startswith('ANS') for word, tag in tagged]):
        continue
    
    for i in range(1, len(tagged)):
        train_words.append(tagged[i][0])
        train_tags.append(tagged[i][1])
        train_previous_words.append(tagged[i-1][0])
        train_previous_tags.append(tagged[i-1][1])
        train_questions.append(q)

In [ ]:
dev_words = []
dev_tags = []
dev_previous_words = []
dev_previous_tags = []
dev_questions = []

for pair in qa_pairs_dev:
    sent = pair[0].replace(' , ',  ', ')
    answers = pair[-1]
    q = pair[2]
    
    tagged = [('<START>', '<START>'), ] + label_text(sent, answers, 'ANS')
    if not any([tag.startswith('ANS') for word, tag in tagged]):
        continue
    
    for i in range(1, len(tagged)):
        dev_words.append(tagged[i][0])
        dev_tags.append(tagged[i][1])
        dev_previous_words.append(tagged[i-1][0])
        dev_previous_tags.append(tagged[i-1][1])
        dev_questions.append(q)

Теперь всё это векторизуем. 

Тэги кодируем через onehot.

In [ ]:
# PREVIOUS TAG ENCODING


lenc_prev_tag = LabelEncoder()
int_prev_tag_enc = lenc_prev_tag.fit_transform(train_previous_tags)
onehot_prev_tag = OneHotEncoder(sparse=True)
int_prev_tag = int_prev_tag_enc.reshape(len(int_prev_tag_enc), 1)

X_prev_tag_train = onehot_prev_tag.fit_transform(int_prev_tag)

int_prev_tag_enc_dev = lenc_prev_tag.transform(dev_previous_tags)
int_prev_tag_dev = int_prev_tag_enc_dev.reshape(
                                     len(int_prev_tag_enc_dev),1)

X_prev_tag_dev = onehot_prev_tag.transform(int_prev_tag_dev)

Слова через CountVectorizer на символах

In [ ]:
# WORD ENCODING
cv_word = CountVectorizer(ngram_range=(2,4), analyzer='char', max_features=5000)
cv_word.fit(train_words)

X_word_train = cv_word.transform(train_words)

X_word_dev = cv_word.transform(dev_words)

In [ ]:
# PREVIOUS WORD ENCODING
cv_prev_word = CountVectorizer(ngram_range=(2,4), analyzer='char', max_features=5000)
cv_prev_word.fit(train_previous_words)

X_prev_word_train = cv_prev_word.transform(train_previous_words)

X_prev_word_dev = cv_prev_word.transform(dev_previous_words)

Вопросы через CountVectorizer на словах

In [ ]:
# QUESTION WORD ENCODING
uniq_questions = set(train_questions)
cv_question = CountVectorizer(max_features=5000)
cv_question.fit(uniq_questions)

X_questions_train = cv_question.transform(train_questions)
X_questions_dev = cv_question.transform(dev_questions)

Скливаем все в одну большую матрицу

In [ ]:

X_train = hstack([X_word_train,
                  X_prev_tag_train,
                  X_prev_word_train,
                  X_questions_train])

In [ ]:

X_dev = hstack([X_word_dev,
                  X_prev_tag_dev,
                  X_prev_word_dev,
                  X_questions_dev])

Обучаем что-то и смотрим на качество.

In [ ]:
clf = LogisticRegressionCV(class_weight='balanced')
clf.fit(X_train, train_tags)

In [ ]:
print(classification_report(dev_tags, clf.predict(X_test_svd)))


Можно проверить как это работает на каком-то конкретном примере.

In [ ]:
def vectorize(word, prev_tag, prev_word, question):
    int_prev_tag_enc = lenc_prev_tag.transform(
                                   [prev_tag])
    int_prev_tag = int_prev_tag_enc.reshape(
                                         len(int_prev_tag_enc),1)

    X_prev_tag = onehot_prev_tag.transform(int_prev_tag)

    X_word = cv_word.transform([word])
    X_prev_word = cv_prev_word.transform([prev_word])
    X_question = cv_question.transform([question])

    X = hstack([X_word,
                  X_prev_tag,
                  X_prev_word,
                  X_question])
    
    pred = clf.predict(X)[0]
    
    return pred

        

In [ ]:
qa_pairs_dev[10]

Задаем предложение и вопрос. Кодируем обученными векторайзерами и предиктим по одному тэгу, передавая предсказанный в следующих шаг.

In [ ]:
sent = qa_pairs_dev[10][0]
q = 'how many Israeli were killed'

sent_tokens = ['<START>'] + sent.split()
tags_pred = ['<START>']

for i in range(1, len(sent_tokens)):
    word = sent_tokens[i]
    prev_word = sent_tokens[i-1]
    prev_tag = tags_pred[i-1]
    
    pred = vectorize(word, prev_tag, prev_word, q)
    
    tags_pred.append(pred)
        


In [ ]:
list(zip(sent_tokens, tags_pred))

### Задание 2.
Попробуйте улучшить модель любыми способами. Если данных слишком много используйте только какую-то часть пар.